In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
def LWR(X, xi, Y, Tau):
	TransposeX = np.transpose(X)
	W = kernel(X, xi, Tau)
	XTransposeW = TransposeX * W
	XTransposeWX = np.matmul(XTransposeW, X)
	InvXTransposeWX = np.linalg.pinv(XTransposeWX)
	InvXTransposeWXXTransposeW = np.matmul(InvXTransposeWX, XTransposeW)
	InvXTransposeWXXTransposeWY = np.matmul(InvXTransposeWXXTransposeW, Y)
	InvXTransposeWXXTransposeWYTranspose = np.transpose(InvXTransposeWXXTransposeWY)
	return InvXTransposeWXXTransposeWYTranspose.dot(xi)


In [3]:
def calculate_error(Y, Y_pred):
    error = 0
    for i in range(len(Y)):
        error += abs(Y[i] - Y_pred[i]) / Y[i]
    error = error / len(Y)
    return error * 100

In [4]:
def kernel(X, xi, Tau):
    return np.exp(-np.sum((xi - X) ** 2, axis = 1) / (2 * Tau * Tau))

In [5]:
input_data = pd.read_csv('Housing Price data set.csv', usecols = ["price", "lotsize", "bedrooms", "bathrms"])
FloorArea = input_data['lotsize']
NoOfBedrooms = input_data['bedrooms']
NoOfBathrooms = input_data['bathrms']
Y = input_data['price']
Y = np.array(Y)
Y = Y.reshape(Y.shape[0], 1)

In [6]:
# Performing feature scanning on FloorArea
FloorArea_Mean = np.mean(FloorArea)
FloorArea_Max = max(FloorArea)
FloorArea_Min = min(FloorArea)
FloorArea_Scaled = []
for i in FloorArea:
	FloorArea_Scaled.append((i - FloorArea_Mean) / (FloorArea_Max - FloorArea_Min))

X = []
for i in range(len(FloorArea)):
	X.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
X = np.array(X)

Tau = 0.00005
print("Using Locally Weighted Linear Regression for Tau = " + str(Tau))
pred = []
for i in range(X.shape[0]):
	y_pred = LWR(X, X[i], Y, Tau)
	pred.append(y_pred)
print("Mean absolute percentage error is : " + str(calculate_error(Y,pred)))
print()

Price = input_data['price']


Using Locally Weighted Linear Regression for Tau = 5e-05
Mean absolute percentage error is : [5.40731789]



In [7]:
#segmenting the features
FeaturesTrain = []
for i in range(383):
	FeaturesTrain.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
PriceTrain = Price[:383]
PriceTest = []
FeaturesTest = []
for i in range(383, len(Price)):
	FeaturesTest.append([1, FloorArea_Scaled[i], NoOfBedrooms[i], NoOfBathrooms[i]])
	PriceTest.append(Price[i])
m = len(FeaturesTrain)


In [8]:
# Function to calculate Slope to find coefficients
def Slope(Coeff, FeaturesTrain, PriceTrain, ind):
	Error = 0
	for i in range(len(FeaturesTrain)):
		itr = 0
		for j in range(len(Coeff)):
			itr = itr + Coeff[j] * FeaturesTrain[i][j]
		Error += (itr - PriceTrain[i]) * FeaturesTrain[i][ind]
	return Error


In [9]:
# Using scaled batch gradient with regularisation
print("Using scaled batch gradient with regularisation")
LearningRate = 0.001
LambdaParameter = -49
Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)
for epochs in range(5000):
	TempCoeff = Coeff.copy()
	for j in range(len(Coeff)):
		if (j == 0):
			TempCoeff[j] = TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))	
		else:
			TempCoeff[j] = (1 - LearningRate * LambdaParameter / m) * TempCoeff[j] - ((LearningRate / m) * (Slope(Coeff, FeaturesTrain, PriceTrain, j)))
	Coeff = TempCoeff.copy()
print("Final coefficients are:")
print(Coeff)

Using scaled batch gradient with regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[5037.585668619078, 11147.667574879839, 10378.580439168689, 22647.298983883848]


In [10]:
# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()


Mean absolute percentage error is : 19.92701396456417



In [11]:
def SlopeStoch(Coeff,FeaturesTrain,ActualVal,ind):
	itr = 0
	for j in range(len(Coeff)):
		itr = itr + Coeff[j]*FeaturesTrain[j]
	return (itr - ActualVal) * FeaturesTrain[ind]


In [12]:
# Using Scaled Stochastic gradient with regularisation.
print("Using Stochastic gradient with regularisation")

# different values of tau were tried to get this as the best among other values
LearningRate = 0.004
LambdaParameter = 142000
Coeff = [0, 0, 0, 0]
print("Initial coefficients: ")
print(Coeff)

for iter in range(10):
	for i in range(len(PriceTrain)):
		TempCoeff = Coeff.copy()
		for j in range(4):
			if j == 0:
				TempCoeff[j] = TempCoeff[j] - (LearningRate * (SlopeStoch(Coeff, FeaturesTrain[i], PriceTrain[i], j)))
			else:
				TempCoeff[j] = (1 - LearningRate * LambdaParameter / m) * TempCoeff[j] - (LearningRate * (SlopeStoch(Coeff, FeaturesTrain[i], PriceTrain[i], j)))
		Coeff = TempCoeff.copy()

print("Final coefficients are:")
print(Coeff)

Using Stochastic gradient with regularisation
Initial coefficients: 
[0, 0, 0, 0]
Final coefficients are:
[68977.37183533033, 153.6672368058788, 622.1158811423422, 207.34938918615728]


In [13]:
# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()


Mean absolute percentage error is : 22.392602067246287



In [14]:
# Using Scaled Minibatch gradient with regularisation for batch size = 20
print("Using Scaled Minibatch gradient with regularisation for batch size = 20")

BatchSize = 20;
LearningRate = 0.002
LambdaParameter = -372
Coeff = [0, 0, 0, 0]
NoOfBatches = math.ceil(len(PriceTrain) / BatchSize)
equallyDiv = False
if (len(PriceTrain) % BatchSize == 0):
	equallyDiv = True;

for epoch in range(30):
	for batch in range(NoOfBatches):
		Summation = [0, 0, 0, 0]
		for j in range(len(Coeff)):
			for i in range(BatchSize):
				if (batch * BatchSize + i == len(FeaturesTrain)):
					break
				PredictedValue = 0.0
				for wj in range(len(Coeff)):
					PredictedValue += Coeff[wj] * FeaturesTrain[batch * BatchSize + i][wj]
				PredictedValue -= PriceTrain[batch * BatchSize + i]
				PredictedValue *= FeaturesTrain[batch * BatchSize + i][j]
				Summation[j] += PredictedValue;

		if (not equallyDiv and batch == NoOfBatches - 1):
			for j in range(len(Summation)):
				if j == 0:
					Coeff[j] -= (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
				else:
					Coeff[j] = (1 - LearningRate * LambdaParameter / m) * Coeff[j] - (Summation[j] / (len(PriceTrain) % BatchSize)) * LearningRate
		else:
			for j in range(len(Summation)):
				if j == 0:
					Coeff[j] -= (Summation[j] / BatchSize) * LearningRate
				else:
					Coeff[j] = (1 - LearningRate * LambdaParameter / m) * Coeff[j] - (Summation[j] / BatchSize) * LearningRate
print("Final coefficients are:")
print(Coeff)


Using Scaled Minibatch gradient with regularisation for batch size = 20
Final coefficients are:
[888.9201243627547, 5168.211726125255, 17701.360814619125, 15202.387873756412]


In [15]:
# Finding Mean absolute percentage error.
Error = 0
for i in range(len(FeaturesTest)):
	predicted = 0
	for j in range(len(Coeff)):
	  	predicted = predicted + Coeff[j] * FeaturesTest[i][j]
	Error += abs(predicted - PriceTest[i]) / PriceTest[i]
Error = (Error / len(FeaturesTest)) * 100
print("Mean absolute percentage error is : " + str(Error))
print()

Mean absolute percentage error is : 19.550681895981263

